# Linear Regression  on a Combined Cycle Power Plant (CCPP) data

## IMPORTANT: make sure to rerun all the code from the beginning to obtain the results for the final version of your notebook, since this is the way we will do it before evaluting your notebook!!!

## Dataset description

The dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. Features consist of hourly average ambient variables Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V) to predict the net hourly electrical energy output (EP)  of the plant.

A combined cycle power plant (CCPP) is composed of gas turbines (GT), steam turbines (ST) and heat recovery steam generators. In a CCPP, the electricity is generated by gas and steam turbines, which are combined in one cycle, and is transferred from one turbine to another. While the Vacuum has effect on the Steam Turbine, the other three of the ambient variables effect the GT performance.

In [ ]:
# to get in-line plots
%matplotlib nbagg
import matplotlib.pyplot as plt

import numpy as np
import scipy as sp
from scipy import stats

## Import Data
Load the data from a .csv file

**TO DO:** insert your ID number (matricola)


In [ ]:
# Load the data
IDnumber = # COMPLETE
np.random.seed(IDnumber)

filename = "data/ccpp_Data_clean.csv"

Data = np.genfromtxt(filename, delimiter=';',skip_header=1)


# A quick overview of data

To inspect the data you can use the method describe()

In [ ]:
dataDescription = stats.describe(Data)
print(dataDescription)

Data.shape

#for more interesting visualization: use Panda!

# Split data in training, validation and test sets



Given $m$ total data, keep $m_t$ data as training data, $m_{val}:=m_{tv}-m_t$ as validation data and $m_{test}:=m - m_{val} - m_t = m-m_{tv}$. For instance one can take $m_t=m/3$ of the data as training, $m_{val}=m/3$  validation and $m_{test}=m/3$ as testing. Let us define as define

$\bullet$ $S_{t}$ the training data set

$\bullet$ $S_{val}$ the validation data set

$\bullet$ $S_{test}$ the testing data set


The reason for this splitting is as follows:

TRAINING DATA: The training data are used to compute the empirical loss
$$
L_S(h) = \frac{1}{m_t} \sum_{z_i \in S_{t}} \ell(h,z_i)
$$
which is used to estimate $h$ in a given model class ${\cal H}$.
i.e. 
$$
\hat{h} = {\rm arg\; min}_{h \in {\cal H}} \, L_S(h)
$$

VALIDATION DATA: When different model classes are present (e.g. of different complexity such as linear regression which uses a different number $d_j$ of regressors $x_1$,...$x_{d_j}$), one has to choose which one is the "best" complexity. 
Let ${\cal H}_{d_j}$ be the space of models as a function of the complexity $d_j$ and let 
$$
\hat h_{d_j} = {\rm arg\; min}_{h \in {\cal H}_{d_j}} \, L_S(h)
$$

One can estimate the generalization error for model $\hat h_{d_j}$ as follows:
$$
L_{{\cal D}}(\hat h_{d_j}) \simeq \frac{1}{m_{val}} \sum_{ z_i \in S_{val}} \ell(\hat h_{d_j},z_i)
$$
and then choose the complexity which achieves the best estimate of the generalization error
$$
\hat d_j: = {\rm arg\; min}_{d_j} \,\frac{1}{m_{val}} \sum_{ z_i \in S_{val}} \ell(\hat h_{d_j},z_i)
$$

TESTING DATA: Last, the test data set can be used to estimate the performance of the final estimated model
$\hat h_{\hat d_j}$ using:
$$
L_{{\cal D}}(\hat h_{\hat d_j}) \simeq \frac{1}{m_{test}} \sum_{ z_i \in S_{test}} \ell(\hat h_{\hat d_j},z_i)
$$


**TO DO**: split the data in training, validation and test sets (suggestion: use $m_t=m_{val} = \lfloor\frac{m}{3}\rfloor$, $m_{test} = m-m_t-m_{val}$)

In [ ]:
#get number of total samples
num_total_samples = Data.shape[0]

print("Total number of samples: ", num_total_samples)

#size of each chunk of data for training, validation, testing
size_chunk = # COMPLETE
print("Size of each chunk of data: ", size_chunk)

#shuffle the data
np.random.shuffle(Data)


#training data  

X_training = # COMPLETE
Y_training = # COMPLETE
print("Training input data size: ", X_training.shape)
print("Training output data size: ", Y_training.shape)

#validation data, to be used to choose among different models
X_validation = # COMPLETE
Y_validation = # COMPLETE
print("Validation input data size: ", X_validation.shape)
print("Validation output data size: ", Y_validation.shape)

#test data, to be used to estimate the true loss of the final model(s)
X_test = # COMPLETE
Y_test = # COMPLETE
print("Test input data size: ", X_test.shape)
print("Test output data size: ", Y_test.shape)

# Data Normalization

It is common practice in Statistics and Machine Learning to scale the data (= each variable) so that it is centered (zero mean) and has standard deviation equal to $1$. This helps in terms of numerical conditioning of the (inverse) problems of estimating the model (the coefficients of the linear regression in this case), as well as to give the same scale to all the coefficients. 

In [ ]:
#scale the data

# standardize the input matrix
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_training)
X_training = scaler.transform(X_training)
print ("Mean of the training input data:", X_training.mean(axis=0))
print ("Std of the training input data:",X_training.std(axis=0))
X_validation = scaler.transform(X_validation) # use the same transformation on validation data
print ("Mean of the validation input data:", X_validation.mean(axis=0))
print ("Std of the validation input data:", X_validation.std(axis=0))
X_test = scaler.transform(X_test) # use the same transformation on test data
print ("Mean of the test input data:", X_test.mean(axis=0))
print ("Std of the test input data:", X_test.std(axis=0))



# Model Training 

The model is trained (= estimated) minimizing the empirical error
$$
L_S(h) := \frac{1}{m_t} \sum_{z_i \in S_{t}} \ell(h,z_i)
$$
When the loss function is the quadratic loss
$$
\ell(h,z) := (y - h(x))^2
$$
we define  the Residual Sum of Squares (RSS) as
$$
RSS(h):= \sum_{z_i \in S_{t}} \ell(h,z_i) = \sum_{z_i \in S_{t}} (y_i - h(x_i))^2
$$ so that the training error becomes
$$
L_S(h) = \frac{RSS(h)}{m_t}
$$

We recal that, for linear models we have $h(x) = <w,x>$ and the Empirical error $L_S(h)$ can be written
in terms of the vector of parameters $w$ in the form
$$
L_S(w) = \frac{1}{m_t} \|Y - X w\|^2
$$
where $Y$ and $X$ are the matrices whose $i-$th row are, respectively, the output data $y_i$ and the input vectors $x_i^\top$.


 **TO DO:** compute the linear regression coefficients implementing the least square algorithm as done in Lab0 and using np.linalg.lstsq from scikitlear 

In [ ]:
#compute linear regression coefficients for training data

#add a 1 at the beginning of each sample for training, validation, and testing
m_training = # COMPLETE LENGTH OF TRAINING DATA

X_training = np.hstack((np.ones((m_training,1)),X_training))
#print(X_training[0,:])

m_validation = # COMPLETE LENGTH OF VALIDATION DATA
X_validation = np.hstack((np.ones((m_validation,1)),X_validation))
#print(X_validation[0,:])

m_test = # COMPLETE LENGTH OF TEST DATA
X_test = np.hstack((np.ones((m_test,1)),X_test))
#print(X_test[0,:])

# Compute the least-squares solution using SVD AND PSEUDOINVERSE AS ABOVE



# ADD CODE

w_hand = # COMPLETE WITH LEAST SQUARES COEFFICIENTS


# Compute the least-squares coefficients using linalg.lstsq

w_np, RSStr_np, rank_Xtr, sv_Xtr = # COMPLETE

print("LS coefficients by hand:", w_hand)

print("LS coefficients with numpy lstsq:", w_np)

# compute Residual sums of squares by hand

RSStr_hand = # COMPLETE

print("RSS by hand:",  RSStr_hand)
print("RSS with numpy lstsq: ", RSStr_np)

print("Empirical risk by hand:", RSStr_hand/m_training)
print("Empirical risk with numpy lstsq:", RSStr_np/m_training)



## Data prediction 

Compute the output predictions on both training and validation set and compute the Residual Sum of Sqaures (RSS) defined above, the Emprical Loss and the quantity $1-R^2$ where
$$
R^2 = \frac{\sum_{z_i \in S_t} (\hat y_i - \bar y)^2}{\sum_{z_i \in S_t} (y_i - \bar y)^2} \quad \quad \bar y = \frac{1}{m_t} \sum_{z_i \in S_t} y_i
$$
is the so-called "Coefficient of determination" (COD)

**TO DO**: Compute these quantities on  training, validation and test sets.


In [ ]:
#compute predictions on training and validation
prediction_training = # COMPLETE
prediction_validation = # COMPLETE
prediction_test = # COMPLETE

#what about the loss for points in the validation data?
RSS_validation = # COMPLETE
RSS_test = # COMPLETE

print("RSS on validation data:",  RSS_validation)
print("Loss estimated from validation data:", RSS_validation/m_validation)


#another measure of how good our linear fit is given by the following (that is 1 - R^2)
Rmeasure_training = # COMPLETE
Rmeasure_validation = # COMPLETE
Rmeasure_test = # COMPLETE

print("Measure on Training Data (1-R^2):", Rmeasure_training)
print("Measure on Validation Data(1-R^2):", Rmeasure_validation)
print("Measure on Test Data(1-R^2):", Rmeasure_test)

## ... and plot:


### (1) output predictions on training  data

In [ ]:
# Plot predictions on Training data 
plt.figure()

#the following is just for nice plotting, not required: it sorts the predictions by value so that they fall on
# a line and it's easier to spot the differences
sorting_permutation = sorted(range(len(prediction_training[0:m_training])), key=lambda k: prediction_training[0:m_training][k])
plt.plot(Y_training[sorting_permutation], 'ko', alpha=0.5)
plt.plot(prediction_training[sorting_permutation], 'rx')

plt.xlabel('Input (index of instance)')
plt.ylabel('Predicted Output')
plt.title('Predictions on Training Data')
plt.show()

### (2) output predictions on validation  data

In [ ]:
# Plot predictions on validation data 
plt.figure()

#the following is just for nice plotting, not required: it sorts the predictions by value so that they fall on
# a line and it's easier to spot the differences
sorting_permutation = sorted(range(len(prediction_validation[0:m_validation])), key=lambda k: prediction_validation[0:m_validation][k])
plt.plot(Y_validation[sorting_permutation], 'ko', alpha=0.5)
plt.plot(prediction_validation[sorting_permutation], 'gx')


plt.xlabel('Input (index of instance)')
plt.ylabel('Predicted Output')
plt.title('Predictions on Validation Data')
plt.show()

## Visualization:  3D plot with fitted hyperplane (only w.r.t. 2 regressors) for training set

To the purpose of visualization we plot the linear function only w.r.t. the first 2 variables (of course any other choice could be made)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.gca(projection='3d')
#plot only m=100 points
m = 100
x1 = X_training[0:m-1,1]
x2 = X_training[0:m-1,2]
ax.scatter(x1,x2,prediction_training[0:m-1], c='r')

#generate a meshgrid to be used to plot surface

#n = number of points to be used in the grid
n=10
x1_surf = np.arange(0.9*min(x1), 1.1*max(x1), (1.1*max(x1)-0.9*min(x1))/n)
x2_surf = np.arange(0.9*min(x2), 1.1*max(x2), (1.1*max(x2)-0.9*min(x2))/n)                

# generate a mesh
x1_surf, x2_surf = np.meshgrid(x1_surf, x2_surf)

#get corresponding input and then the predicted output from linear regression model
x_surf = np.hstack((np.ones((n*n,1)),list(zip( x1_surf.ravel(),x2_surf.ravel()  ))))
y_surf = np.dot(x_surf,w_np)

ax.plot_surface(x1_surf,x2_surf,y_surf.reshape(n,n), alpha=0.5, color=[1,1,1])

plt.show()

## Ordinary Least-Squares using scikit-learn

A fast way to compute the LS estimate is through sklearn.linear_model

In [ ]:
# Remove the ``ones'' column in the features matrix (sklearn inserts it automatically)
X_training = X_training[:,1:]
X_test = X_test[:,1:]


In [ ]:
from sklearn import linear_model
LinReg = linear_model.LinearRegression()  # build the object LinearRegression
LinReg.fit(X_training, Y_training)  # estimate the LS coefficients
print("Intercept:", LinReg.intercept_)
print("Least-Squares Coefficients:", LinReg.coef_)
prediction_training = LinReg.predict(X_training)  # predict output values on training set
prediction_test = LinReg.predict(X_test)  # predict output values on test set
print("Measure on training data:", 1-LinReg.score(X_training, Y_training))
